In [16]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

from IPython.display import display
%matplotlib inline

In [17]:
# Read student data
loan_data = pd.read_csv("loan.csv", low_memory=False)
print ("Loan data read successfully!")

Loan data read successfully!


In [18]:
print (loan_data.head())

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501    1296599     5000.0       5000.0           4975.0   36 months   
1  1077430    1314167     2500.0       2500.0           2500.0   60 months   
2  1077175    1313524     2400.0       2400.0           2400.0   36 months   
3  1076863    1277178    10000.0      10000.0          10000.0   36 months   
4  1075358    1311748     3000.0       3000.0           3000.0   60 months   

   int_rate  installment grade sub_grade     ...      total_bal_il il_util  \
0     10.65       162.87     B        B2     ...               NaN     NaN   
1     15.27        59.83     C        C4     ...               NaN     NaN   
2     15.96        84.33     C        C5     ...               NaN     NaN   
3     13.49       339.31     C        C1     ...               NaN     NaN   
4     12.69        67.79     B        B5     ...               NaN     NaN   

  open_rv_12m  open_rv_24m max_bal_bc all_util total_rev_hi_li

In [46]:
loan_outcome_data = loan_data.drop(loan_data[(loan_data.loan_status != 'Fully Paid') & (loan_data.loan_status != 'Charged Off')].index)
#display(loan_outcome_data.head())

#pif = (loan_outcome_data[loan_outcome_data['loan_status'] == 'Fully Paid'].count()['id'])
#chg = (loan_outcome_data[loan_outcome_data['loan_status'] == 'Charged Off'].count()['id']) 
#print('There are %.0f loans that paid in full, and %.0f loans that were charged off' % (pif, chg))

In [52]:
# TODO: Calculate number of records
n_loans = float(len(loan_outcome_data))

# TODO: Calculate number of features
n_features = float(len(loan_outcome_data.columns) - 1)

# TODO: Calculate paid in full loans
n_pif = float(sum(1 for item in loan_outcome_data['loan_status'] if item==('Fully Paid')))

# TODO: Calculate charged off loans
n_chargeoff = float(sum(1 for item in loan_outcome_data['loan_status'] if item==('Charged Off')))

# TODO: Calculate successful loan rate
success_rate = n_pif / n_loans * 100

# Print the results
print("Total number of loans: {}".format(n_loans))
print("Number of features: {}".format(n_features))
print("Number of loans paid in full: {}".format(n_pif))
print("Number of charge-offs: {}".format(n_chargeoff))
print("Success rate of loans: {:.2f}%".format(success_rate))

Total number of loans: 252971.0
Number of features: 73.0
Number of loans paid in full: 207723.0
Number of charge-offs: 45248.0
Success rate of loans: 82.11%


In [45]:
# Store the 'loan_status' feature in a new variable and remove it from the dataset
outcomes = loan_outcome_data['loan_status']
data = loan_outcome_data.drop('loan_status', axis = 1)

# Show the new dataset with 'loan_status' removed
display(data.head())
display(outcomes.head())

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0     Fully Paid
1    Charged Off
2     Fully Paid
3     Fully Paid
5     Fully Paid
Name: loan_status, dtype: object